In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


In [ ]:
# データベースをSilver_LH_NYCに設定
spark.sql("USE Silver_LH_NYC")

# データベース内のすべてのテーブル名を取得
tables = spark.sql("SHOW TABLES").select("tableName").rdd.flatMap(lambda x: x).collect()

# テーブル名リストを表示
print("List of tables in Silver_LH_NYC:")
for table_name in tables:
    print(table_name)

# すべてのテーブルを結合するデータフレームを作成
fact_tripdf = None

for table_name in tables:
    df = spark.table(table_name)
    if fact_tripdf is None:
        fact_tripdf = df
    else:
        fact_tripdf = fact_tripdf.union(df)

In [ ]:
# 実施した内容が正確に反映されていることを確認
# 結合されたデータフレームの表示
display(fact_tripdf.describe())

In [ ]:
# Table として保存
fact_tripdf.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("Gold_LH_NYC.fact_trip")

In [ ]:
# 既存のテーブルを削除
spark.sql("DROP TABLE IF EXISTS Gold_LH_NYC.dim_weekday")
# データの準備
data = [
    (1, "Sunday"),
    (2, "Monday"),
    (3, "Tuesday"),
    (4, "Wednesday"),
    (5, "Thursday"),
    (6, "Friday"),
    (7, "Saturday")
]

# スキーマの定義
schema = StructType([
    StructField("pickup_weekday", IntegerType(), True),
    StructField("description", StringType(), True)
])
# データフレームの作成
dim_weekday_df = spark.createDataFrame(data, schema)
# テーブルとして保存
dim_weekday_df.write.mode("overwrite").saveAsTable("Gold_LH_NYC.dim_weekday")

In [ ]:
df = spark.sql("SELECT * FROM Gold_LH_NYC.dim_weekday")
display(df)

In [ ]:
# 既存のテーブルを削除
spark.sql("DROP TABLE IF EXISTS Gold_LH_NYC.dim_paymenttype")

# データの準備
data = [
    (1, "Credit card"),
    (2, "Cash"),
    (3, "No charge"),
    (4, "Dispute"),
    (5, "Unknown"),
    (6, "Voided trip")
]

# スキーマの定義
schema = StructType([
    StructField("payment_type", IntegerType(), True),
    StructField("description", StringType(), True)
])

# データフレームの作成
dim_paymenttypedf = spark.createDataFrame(data, schema)

# テーブルとして保存
dim_paymenttypedf.write.mode("overwrite").saveAsTable("Gold_LH_NYC.dim_paymenttype")

In [ ]:
import requests

csv_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"
response = requests.get(csv_url)

with open('/lakehouse/default/Files/taxi_zone_lookup.csv', 'wb') as file:
    file.write(response.content)

In [ ]:
# スキーマの定義
schema = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Borough", StringType(), True),
    StructField("Zone", StringType(), True),
    StructField("service_zone", StringType(), True)
])
# CSVファイルの読み込み
dim_location = spark.read.format("csv").schema(schema).option("header", "true").load("Files/taxi_zone_lookup.csv")
# Deltaテーブルとして保存
dim_location.write.format("delta").mode("overwrite").saveAsTable("Gold_LH_NYC.dim_location")


In [ ]:
display(spark.sql("SELECT * FROM Gold_LH_NYC.dim_location ORDER BY LocationID ASC"))